In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
data_ori = pd.read_csv("data/total_data.csv")

In [3]:
data_ori.head()

,timestamp,submit_id,user_id,problem_id,result_state,memory,running_time,language,code_length,__
0,2022-12-16 17:08:57,52731042.0,jsh00325,14562.0,맞았습니다!!,2020.0,0.0,C++11,592.0,8일 전
1,2022-12-16 15:28:07,52726708.0,teddybear1024,14562.0,맞았습니다!!,2024.0,0.0,C++17,875.0,8일 전
2,2022-12-15 01:15:34,52680839.0,mujigae,14562.0,맞았습니다!!,2156.0,0.0,C++17,554.0,9일 전
3,2022-12-12 15:23:46,52595252.0,wnsgml7267,14562.0,맞았습니다!!,34096.0,68.0,Python 3,420.0,12일 전
4,2022-12-01 00:29:05,52271216.0,gustjs5419,14562.0,맞았습니다!!,2020.0,4.0,C++17,712.0,23일 전


# Null 채우기

## Timestamp & User_Id & Code_length

In [5]:
data=data_ori.copy()
data=data.drop(axis=1, columns=['__'])
data = data.drop(data.loc[data['timestamp'].isna()].index)
data = data.drop(data.loc[data['user_id'].isna()].index)
data = data.drop(data.loc[data['code_length'].isna()].index)

In [6]:
data.isna().sum()

timestamp              0
submit_id              0
user_id                0
problem_id             0
result_state           0
memory          13207832
running_time    13207832
language               0
code_length            0
dtype: int64

## Memory & Running Time

In [7]:
# # fill na : mean
# dic_mean = {}
# dic_runtime = {}
# memory_list = []
# running_time_list = []
# data['memory_dummy'] = data['memory']
# data['running_time_dummy'] = data['running_time']
# for _, grouped in tqdm(problem_lang_memo_running_mean):
#     problem_id, language = _
#     memory, running_time = grouped.mean()
#     dic_mean[f'(data["problem_id"]=={problem_id}) & (data["language"]=={language}) & (data["memory"].isna())'] = memory
#     dic_runtime[f'(data["problem_id"]=={problem_id}) & (data["language"]=={language}) & (data["running_time"].isna())'] = running_time
#     # data.loc[(data['problem_id']==problem_id) & (data['language']==language) & (data['memory'].isna())] = memory
#     # data.loc[(data['problem_id']==problem_id) & (data['language']==language) & (data['running_time'].isna())] = running_time


NameError: name 'tqdm' is not defined

In [ ]:
# # fill na : -1
# data['memory']=data['memory'].fillna(-1)
# data['running_time']=data['running_time'].fillna(-1)

# 수치화

## Result state

In [8]:
print(data['result_state'].nunique())
data['result_state'].unique()

1728


array(['맞았습니다!!', '메모리 초과', '런타임 에러\xa0(FileNotFoundError)', ...,
       '2147483647점', '580245801점', '123456789점'], dtype=object)

In [9]:
data['result_state'] = data['result_state'].apply(lambda x: '맞았습니다!!' if x.endswith('점') and float(x[:-1]) >= 100 else x)
data['result_state'] = data['result_state'].apply(lambda x: '틀렸습니다' if x.endswith('점') and float(x[:-1]) < 100 else x)
data['result_state'] = data['result_state'].apply(lambda x: '런타임 에러' if x.startswith('런타임') else x)
data['result_state'] = data['result_state'].apply(lambda x: '틀렸습니다' if x.startswith('맞았습니다!!\xa0(') else x)
data['result_state'] = data['result_state'].apply(lambda x: '맞았습니다' if x.startswith('맞') else x)

data = data.loc[~data['result_state'].isin(['채점 불가', '채점 준비 중', '채점 중', '삭제된 제출'])]

print(data['result_state'].nunique())
data['result_state'].unique()

8


array(['맞았습니다', '메모리 초과', '런타임 에러', '틀렸습니다', '컴파일 에러', '출력 초과', '시간 초과',
       '출력 형식이 잘못되었습니다'], dtype=object)

## language

In [10]:
print(data['language'].nunique())
data['language'].unique()

89


array(['C++11', 'C++17', 'Python 3', 'Java 11', 'node.js', 'PyPy3',
       'C++14', 'C++20', 'C++20 (Clang)', 'Java 8', 'Swift', 'C++98',
       'Java 8 (OpenJDK)', 'Kotlin (JVM)', 'C2x', 'Java 15', 'C11', 'C99',
       'C++17 (Clang)', 'Rust 2021', 'Go', 'C#', 'TypeScript', 'Lua',
       'Golfscript', '아희', 'Rust 2018', 'Ruby', 'PHP', 'C++11 (Clang)',
       'C# 6.0 (Mono)', 'C++14 (Clang)', 'C++98 (Clang)', 'Python 2', 'D',
       'Scala', 'Text', 'F#', 'Rust 2015', 'C11 (Clang)', 'Go (gccgo)',
       'C90', 'OCaml', 'PyPy2', 'Kotlin (Native)', 'F# (Mono)',
       'C99 (Clang)', 'Assembly (32bit)', 'Cobra', 'Perl', 'Pascal',
       'D (LDC)', 'Fortran', 'Scheme', 'C2x (Clang)', 'Objective-C++',
       'Befunge', 'Haskell', 'R', 'C# 3.0 (Mono)', 'Ruby 1.9', 'Clojure',
       'Ruby 1.8', 'Rhino', 'SystemVerilog', '엄준식', 'VB.NET 4.0 (Mono)',
       'VB.NET 2.0 (Mono)', 'awk', 'Assembly (64bit)', 'Bash',
       'Whitespace', 'Brainf**k', 'INTERCAL', 'LOLCODE', 'Objective-C',
       'Tcl'

In [11]:
def integrate_language(df, col):
    df.loc[df['language'].str.startswith(col), 'language'] = col
    print(col , 'complete')
    return df
integrate_lang = ['C++', 'Python', 'Java', 'C#', 'Kotlin', 'Rust', 'Ruby']

for lang in integrate_lang:
    data = integrate_language(data, lang)

# C
data.loc[(data['language'].str.startswith('C99')) | (data['language'].str.startswith('C90') | (data['language'].str.startswith('C11')) | (data['language'].str.startswith('C2x')) | (data['language'].str.startswith('C2x (Clang)')) ), 'language'] = 'C'

# Python
data.loc[(data['language'].str.startswith('PyPy3')), 'language'] = 'Python'

C++ complete
Python complete
Java complete
C# complete
Kotlin complete
Rust complete
Ruby complete


In [12]:
print(data['language'].nunique())
data['language'].unique()

61


array(['C++', 'Python', 'Java', 'node.js', 'Swift', 'Kotlin', 'C', 'Rust',
       'Go', 'C#', 'TypeScript', 'Lua', 'Golfscript', '아희', 'Ruby', 'PHP',
       'D', 'Scala', 'Text', 'F#', 'Go (gccgo)', 'OCaml', 'PyPy2',
       'F# (Mono)', 'Assembly (32bit)', 'Cobra', 'Perl', 'Pascal',
       'D (LDC)', 'Fortran', 'Scheme', 'Objective-C++', 'Befunge',
       'Haskell', 'R', 'Clojure', 'Rhino', 'SystemVerilog', '엄준식',
       'VB.NET 4.0 (Mono)', 'VB.NET 2.0 (Mono)', 'awk',
       'Assembly (64bit)', 'Bash', 'Whitespace', 'Brainf**k', 'INTERCAL',
       'LOLCODE', 'Objective-C', 'Tcl', 'Visual Basic', 'Pike', 'bc',
       'Ada', 'Haxe', 'sed', 'SpiderMonkey', 'Groovy', 'Cobol', 'Boo',
       'Nemerle'], dtype=object)

In [13]:
#7000번 이하로 제출된 언어 -> others

lang_value_cnt = data['language'].value_counts().to_frame()
other_langs = lang_value_cnt.loc[lang_value_cnt['language'] < 7000].index
data.loc[data['language'].isin(other_langs), 'language'] = 'others'
data['language'].value_counts()

C++        10211132
Python      5537596
Java        3941102
C           1176433
node.js      234266
Kotlin       107876
Swift         92852
C#            85498
others        33131
Go            30863
Rust          21831
Name: language, dtype: int64

## User_id, Problem_id, submit_id

In [14]:
# mapping dicts

# user to idx & idx to user
user2idx = {user : idx for idx, user in enumerate(data['user_id'].unique())}
idx2user = {idx : user for user, idx in user2idx.items()}

# problem_id to idx & idx to problem_id
problem2idx = {int(problem) : idx for idx, problem in enumerate(data['problem_id'].unique())}
idx2problem = {idx : problem_id for problem_id, idx in problem2idx.items()}

# result state to idx & idx to result state
state2idx = {state : idx for idx, state in enumerate(data['result_state'].unique())}
idx2state = {idx : state for state, idx in state2idx.items()}

# submit_id to idx & idx to submit_id
submit2idx = {int(submit) : idx for idx, submit in enumerate(sorted(data['submit_id'].unique()))}
idx2submit = {idx : submit for submit, idx in submit2idx.items()}

# user to idx & idx to user
lang2idx = {lang : idx for idx, lang in enumerate(data['language'].unique())}
idx2lang = {idx : lang for lang, idx in lang2idx.items()}

In [15]:
for col,colmap in zip(['user_id','problem_id','result_state','submit_id','language'],[user2idx,problem2idx,state2idx,submit2idx,lang2idx]):
    data[col]=data[col].map(colmap)

In [16]:
data.head()

,timestamp,submit_id,user_id,problem_id,result_state,memory,running_time,language,code_length,memory_dummy,running_time_dummy
0,2022-12-16 17:08:57,21372924,0,0,0,2020.0,0.0,0,592.0,2020.0,0.0
1,2022-12-16 15:28:07,21371099,1,0,0,2024.0,0.0,0,875.0,2024.0,0.0
2,2022-12-15 01:15:34,21352943,2,0,0,2156.0,0.0,0,554.0,2156.0,0.0
3,2022-12-12 15:23:46,21319086,3,0,0,34096.0,68.0,1,420.0,34096.0,68.0
4,2022-12-01 00:29:05,21189232,4,0,0,2020.0,4.0,0,712.0,2020.0,4.0


In [17]:
data.to_csv('preprocessed_total_data.csv')